In [3]:
from ultralytics import YOLO

# Load your trained YOLOv9 model (best.pt)
model = YOLO('best.pt')

In [6]:
from deep_sort_realtime.deepsort_tracker import DeepSort


In [8]:
tracker = DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0, max_cosine_distance=0.3)


In [11]:
# ==========================
# Install dependencies
# ==========================
!!pip install ultralytics deep-sort-realtime pyngrok flask opencv-python-headless

# ==========================
# Imports and setup
# ==========================
from flask import Flask, Response, render_template_string
from pyngrok import ngrok
import cv2
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import threading

# ==========================
# Initialize YOLOv9 + DeepSORT
# ==========================
model = YOLO('best.pt')
model.eval()

# Initialize DeepSORT tracker
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    max_cosine_distance=0.3
)

# ==========================
# Flask App
# ==========================
app = Flask(__name__)
from pyngrok import ngrok

port = 5000
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}")

# ==========================
# Detection + Tracking Function
# ==========================
def detect_and_alert(frame):
    # Run YOLOv9 detection
    results = model.predict(frame, verbose=False)[0]
    detections = []

    for result in results.boxes:
        box = result.xyxy[0].cpu().numpy()
        cls = int(result.cls[0].cpu().numpy())
        conf = float(result.conf[0].cpu().numpy())
        detections.append(([box[0], box[1], box[2], box[3]], conf, cls))

    # Update DeepSORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Draw bounding box + ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    return frame

# ==========================
# Frame Generator for Streaming
# ==========================
def generate_frames():
    cap = cv2.VideoCapture('test_video.mp4')

    while True:
        success, frame = cap.read()
        if not success:
            break

        frame = detect_and_alert(frame)
        _, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

# ==========================
# Flask Routes
# ==========================
@app.route('/')
def index():
    return render_template_string('''
        <h2>YOLOv9 + DeepSORT Object Detection (Colab)</h2>
        <img src="{{ url_for('video_feed') }}" width="720">
    ''')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

# ==========================
# Run Flask App
# ==========================
if __name__ == '__main__':
    app.run()


ERROR:pyngrok.process.ngrok:t=2025-11-13T06:05:15+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-13T06:05:15+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.